<a href="https://colab.research.google.com/github/Vadim-sys-create/TTS/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled19_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install gtts

In [ ]:
!pip install --force-reinstall gtts

  Using cached gTTS-2.5.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached gTTS-2.5.4-py3-none-any.whl (29 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (143 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Unin

In [1]:
!pip install flask

In [5]:
!python app.py

2025-03-02 10:33:38.475346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740911618.624095    2442 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740911618.647402    2442 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 10:33:38.722348: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/app.py", line 4, in <module>
    from fuzzywuzzy import process
ModuleNot

In [6]:
!pip install aiogram transformers gtts fuzzywuzzy langdetect nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.8/612.8 kB 30.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f146ba80352102d4c8a413edc047c549ba7549f63802962414bed110e02e8612
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [8]:
text = "Привет, это тестовое аудио."

In [9]:
# Установка библиотек
!pip install aiogram transformers gtts fuzzywuzzy langdetect nest_asyncio

# Импорт библиотек
import os
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from gtts import gTTS
from fuzzywuzzy import process
from langdetect import detect
import nest_asyncio

# Применяем nest_asyncio для работы в Colab
nest_asyncio.apply()

# Загрузка модели и токенизатора
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Ваши ответы на вопросы
my_responses = {
    "Как тебя зовут?": "Меня зовут Аватар, и я здесь, чтобы помочь тебе!",
    "What is your name?": "My name is Avatar, and I'm here to help you!",
    "¿Cómo te llamas?": "Me llamo Avatar, y estoy aquí para ayudarte.",
    "Comment tu t'appelles?": "Je m'appelle Avatar, et je suis là pour vous aider.",
}

# Функция для генерации ответа
def generate_response(question, max_length=50):
    # Поиск наиболее похожего вопроса в словаре
    match, score = process.extractOne(question, my_responses.keys())

    # Если совпадение достаточно точное (например, score > 80), возвращаем ответ
    if score > 80:
        return my_responses[match]

    # Если совпадение неточное, генерируем ответ с помощью модели
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Функция для определения языка текста
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'ru'  # По умолчанию используем русский язык

# Функция для создания аудио из текста
def create_audio_from_text(text, output_file="response.mp3"):
    lang = detect_language(text)  # Автоматически определяем язык
    tts = gTTS(text, lang=lang)
    tts.save(output_file)
    return output_file

# Инициализация бота
API_TOKEN = '7954916275:AAEoTnpSPG_DRajIGC859AqOmVejyrn7VL0'  # Замените на ваш токен
bot = Bot(token=API_TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher()

# Гифки и картинки для ответов
happy_gif = "https://media.giphy.com/media/3o7aTskHEUdgCQAXde/giphy.gif"  # Пример гифки
thinking_image = "https://example.com/thinking.jpg"  # Пример картинки

# Обработчик команды /start
@dp.message(Command("start"))
async def send_welcome(message: types.Message):
    await message.answer("Привет! Я Аватар. Задай мне вопрос!")
    await message.answer_animation(happy_gif)  # Отправляем гифку

# Обработчик текстовых сообщений
@dp.message()
async def echo(message: types.Message):
    user_input = message.text

    if user_input.lower() in ["выход", "exit", "quit"]:
        await message.answer("Аватар: Пока!")
        return

    try:
        # Генерация ответа
        response = generate_response(user_input)
        await message.answer(f"Аватар: {response}")

        # Создание аудио
        audio_file = create_audio_from_text(response)

        # Проверка, создан ли аудиофайл
        if os.path.exists(audio_file):
            print("Аудиофайл создан.")  # Для отладки

            # Отправка аудио (используем путь к файлу)
            with open(audio_file, "rb") as audio:
                await message.answer_audio(audio)
        else:
            print("Ошибка: аудиофайл не создан.")  # Для отладки
            await message.answer("Не удалось создать аудио.")

        # Отправка гифки или картинки
        if "радост" in response.lower():
            await message.answer_animation(happy_gif)  # Отправляем гифку
        else:
            await message.answer_photo(thinking_image)  # Отправляем картинку

        # Удаление временного файла
        if os.path.exists(audio_file):
            os.remove(audio_file)
    except Exception as e:
        await message.answer(f"Произошла ошибка: {str(e)}")  # Исправлено: str(e) для корректного вывода ошибки

# Запуск бота
async def start_bot():
    await dp.start_polling(bot)

# Запуск бота в Colab
await start_bot()

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `d

Аудиофайл создан.


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
ERROR:aiogram.event:Cause exception while process update id=995587229 by bot id=7954916275
TelegramBadRequest: Telegram server says - Bad Request: can't parse entities: Unsupported start tag "_io.bufferedreader" at byte offset 173
Traceback (most recent call last):
  File "<ipython-input-9-d12a11f58cce>", line 106, in echo
    await message.answer_audio(audio)
          ^^^^^^^^^^^^^^^^^^^^^^

Аудиофайл создан.


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
ERROR:aiogram.event:Cause exception while process update id=995587230 by bot id=7954916275
TelegramBadRequest: Telegram server says - Bad Request: can't parse entities: Unsupported start tag "_io.bufferedreader" at byte offset 173
Traceback (most recent call last):
  File "<ipython-input-9-d12a11f58cce>", line 106, in echo
    await message.answer_audio(audio)
          ^^^^^^^^^^^^^^^^^^^^^^

Аудиофайл создан.


ERROR:aiogram.event:Cause exception while process update id=995587231 by bot id=7954916275
TelegramBadRequest: Telegram server says - Bad Request: can't parse entities: Unsupported start tag "_io.bufferedreader" at byte offset 173
Traceback (most recent call last):
  File "<ipython-input-9-d12a11f58cce>", line 106, in echo
    await message.answer_audio(audio)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/aiogram/types/message.py", line 1022, in answer_audio
    return SendAudio(
           ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydantic/main.py", line 214, in __init__
    validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pydantic_core._pydantic_core.ValidationError: 2 validation errors for SendAudio
audio.is-instance[InputFile]
  Input should be an instance of InputFile [type=is_instance_of, input_va

Аудиофайл создан.


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
ERROR:aiogram.event:Cause exception while process update id=995587232 by bot id=7954916275
TelegramBadRequest: Telegram server says - Bad Request: can't parse entities: Unsupported start tag "_io.bufferedreader" at byte offset 173
Traceback (most recent call last):
  File "<ipython-input-9-d12a11f58cce>", line 106, in echo
    await message.answer_audio(audio)
          ^^^^^^^^^^^^^^^^^^^^^^

Аудиофайл создан.
